# Fairness in binary classification [draft]

Study and reproductibility of the paper:
[Leveraging Labeled and Unlabeled data for consistent fair binary classification](https://proceedings.neurips.cc/paper_files/paper/2019/file/ba51e6158bcaf80fd0d834950251e693-Paper.pdf)


Other references:

- [Fairness Beyond Disparate Treatment & Disparate Impact, Zafar & al (2017)](https://arxiv.org/abs/1610.08452)

- [Equal Opportunity in Supervised Learning, Hardt & al (2016)](https://arxiv.org/abs/1610.02413)

- [Empirical Risk Minimization under Fairness Constraints, Donini & al (2020)](https://arxiv.org/abs/1802.08626)


## Loading libraries and data

In [1]:
from collections import namedtuple
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [2]:
from fairness.utils import *
from fairness.data import *

In [3]:
from tabulate import tabulate
from itertools import product

## Estimating results from standard models

Expected output:

<img src='paper_results.png' width=70% height=70%>


In [4]:
C_span = np.logspace(-2, 4, 30)

lin_params = np.asarray([{'C': c} for c in C_span])
rbf_params = np.asarray([{'C':x, 'gamma': y} for x, y in product(C_span, [1e-3, 1e-2, 1e-1, 1])])

In [5]:
DataConfig = namedtuple('DataConfig', 'name load loader_kwargs subsample sample_proportion')
ModelConfig = namedtuple('ModelConfig', 'name need_cv Model model_kwargs cv_config')

data_configs = [
    # name, function, kwargs, subsample_training, subsampling proportion
    DataConfig('Adult', load_adult_data, dict(onehot=False), True, 1),
    # DataConfig('Arrythmia', load_arrhythmia_data, dict(), True, 1),
    # DataConfig('Drug', load_drug_data, dict(standardize=False), True, 1),
    # DataConfig('German', load_german_data, dict(onehot=False, standardize=False), True, 1)
]

model_configs = [
    # name, need_cross_val, model_function, model_kwargs, cv_kwargs
    ModelConfig('Lin.LR', False, LogisticRegression, dict(max_iter=2000, penalty='l2'), lin_params),
    # ModelConfig('Lin.SVM', True, SVC, dict(kernel='linear', probability=True), lin_params),
    # ModelConfig('SVM', True, SVC, dict(kernel='rbf', probability=True), rbf_params),
    # ModelConfig('RF', True, RandomForestClassifier, dict(n_estimators=500), )

]

In [6]:
nb_iter = 1  #  30
scores = ['ACC', 'DEO']
datasets = ['Arrythmia', 'Adult', 'German', 'Drug']

methods = [
    'Lin.SVM', 'Lin.SVM+Ours', 'Lin.LR', 'Lin.LR+Ours', 
    'SVM', 'SVM+Ours', 'LR', 'LR+Ours', 'RF', 'RF+Ours'
]
summary = pd.DataFrame(0, index=methods, columns=pd.MultiIndex.from_product([datasets, scores]))

In [7]:
for mc in model_configs:
    
    print(f'Model: {mc.name}')
    print('------------------------------------------------------')
    for dc in data_configs:
        print('\n***')
        print(f'Processing {dc.name}\n')

        acc = []
        deo = []
        
        calib_acc = []
        calib_deo = []
        
        for i in range(nb_iter):
            # Loading and subsampling the data
            X, y, Xt, yt, s = dc.load(**dc.loader_kwargs)
            
            accs = np.zeros(len(mc.cv_config))
            deos = np.zeros(len(mc.cv_config))

            for j, p in enumerate(mc.cv_config):
                print(f'Iteration {i+1}  Param {j+1}', end='\r')
                current_model = mc.Model(**p, **mc.model_kwargs)
                test_acc = 0
                test_deo = 0
                for train, test in folds_generator(X, y, s):
                    X_train, y_train = X.iloc[train], y.iloc[train]
                    X_test, y_test = X.iloc[test], y.iloc[test]
                    current_model.fit(X_train, y_train)
                    y_pred = current_model.predict(X_test)
                    test_acc += np.mean(y_pred == y_test)
                    test_deo += empirical_unfairness(y_test, y_pred, X_test[s])
                
                accs[j] = test_acc / 10
                deos[j] = test_deo / 10
    
            best_params = get_npv_optimal_params(mc.cv_config, accs, deos, threshold=.99)
            model = mc.Model(**best_params, **mc.model_kwargs).fit(X, y)

            
            y_pred = model.predict(Xt)
            y_prob = model.predict_proba(Xt)[:, 1]
            
            acc.append(np.mean(y_pred == yt))
            deo.append(empirical_unfairness(yt, y_pred, Xt[s]))

            theta, value, y_calib = recalibrate_predictions(y_prob, Xt[s])
            calib_acc.append(np.mean(y_calib == yt))
            calib_deo.append(empirical_unfairness(yt, y_calib, Xt[s]))


        
        acc_avg = np.nanmean(acc)
        acc_std = np.nanstd(acc)
        
        deo_avg = np.nanmean(deo)
        deo_std = np.nanstd(deo)

        calib_acc_avg = np.nanmean(calib_acc)
        calib_acc_std = np.nanstd(calib_acc)
        
        calib_deo_avg = np.nanmean(calib_deo)
        calib_deo_std = np.nanstd(calib_deo)
        
        summary.loc[mc.name, (dc.name, 'ACC')] = f'{acc_avg:.2f} ± {acc_std:.2f}'
        summary.loc[mc.name, (dc.name, 'DEO')] = f'{deo_avg:.2f} ± {deo_std:.2f}'
        summary.loc[mc.name+'+Ours', (dc.name, 'ACC')] = f'{calib_acc_avg:.2f} ± {calib_acc_std:.2f}'
        summary.loc[mc.name+'+Ours', (dc.name, 'DEO')] = f'{calib_deo_avg:.2f} ± {calib_deo_std:.2f}'
       
        print(
            tabulate(
                [
                    ['Original', f'{acc_avg:.2f} ± {acc_std:.2f}', f'{deo_avg:.2f} ± {deo_std:.2f}'], 
                    ['Calibrated', f'{calib_acc_avg:.2f} ± {calib_acc_std:.2f}', f'{calib_deo_avg:.2f} ± {calib_deo_std:.2f}']
                ], 
                headers=['Estimation', 'Accuracy', 'Unfairness'],
                tablefmt='grid',
            )
        )
        
    print('------------------------------------------------------')


Model: Lin.LR
------------------------------------------------------

***
Processing Adult

+--------------+-------------+--------------+
| Estimation   | Accuracy    | Unfairness   |
+==============+=============+==============+
| Original     | 0.84 ± 0.00 | 0.15 ± 0.00  |
+--------------+-------------+--------------+
| Calibrated   | 0.84 ± 0.00 | 0.05 ± 0.00  |
+--------------+-------------+--------------+
------------------------------------------------------
